In [ ]:
!mkdir /content/MIMICData

In [ ]:
!unzip /content/datafr.zip -d /content/MIMICData

Archive:  /content/datafr.zip
  inflating: /content/MIMICData/datafr/AI.txt  
  inflating: /content/MIMICData/datafr/textAI.txt  


In [ ]:
!cat /content/MIMICData/datafr/textAI.txt

On utilise le terme �d�intelligence artificielle� ou d�IA pour d�signer les ordinateurs et programmes informatiques capables de performances habituellement associ�es � l�intelligence humaine. Par exemple, la capacit� � interagir avec l�homme, � traiter de grandes quantit�s de donn�es ou encore � apprendre progressivement et donc � s�am�liorer de mani�re continue. C�est donc un vaste sujet, en perp�tuelle �volution !
L�intelligence artificielle peut se d�finir comme � l�ensemble de th�ories et de techniques mises en �uvre en vue de r�aliser des machines capables de simuler l�intelligence �, selon le Larousse. Soit des ordinateurs ou des programmes avec des puissances de calcul capables de performances habituellement associ�es � l�intelligence humaine, et amplifi�es par la technologie : 
- Capacit� de raisonner
- Capacit� de traiter de grandes quantit�s de donn�es
- Facult� de discerner des patterns et des mod�les ind�tectables par un humain
- Aptitude � comprendre et analyser ces mod�le

In [ ]:
# Install the latest release of Haystack in your own environment 
#! pip install farm-haystack

!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-br248t69/farm-haystack_7f463f8061924a829d25a90fc46ae774
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-br248t69/farm-haystack_7f463f8061924a829d25a90fc46ae774
  Resolved https://github.com/deepset-ai/haystack.git to commit 15203d864be9bbe5f39585a9081a940e7101abb5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached azure_ai_formrecognizer-3.2.0-py3-none-any.whl (228 kB)
  Using cached posthog-2.2.0-py2.py3-none-any.whl (33 kB)
  Using cached quantulum3-0.8.0-py3-none-any.whl (10.7 MB)
  Using cached langdetect-1.0.9-py3-none-any.whl
  Using cached seqeval-1.

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable  HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://docs.haystack.deepset.ai/docs/telemetry


In [ ]:
doc_dir = "/content/MIMICData"


# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)


INFO:haystack.utils.preprocessing:Converting /content/MIMICData/datafr/textAI.txt
INFO:haystack.utils.preprocessing:Converting /content/MIMICData/datafr/AI.txt


In [ ]:
print(docs[:1])

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

[<Document: {'content': "On utilise le terme dintelligence artificielle ou dIA pour dsigner les ordinateurs et programmes informatiques capables de performances habituellement associes  lintelligence humaine. Par exemple, la capacit  interagir avec lhomme,  traiter de grandes quantits de donnes ou encore  apprendre progressivement et donc  samliorer de manire continue. Cest donc un vaste sujet, en perptuelle volution !\nLintelligence artificielle peut se dfinir comme  lensemble de thories et de techniques mises en uvre en vue de raliser des machines capables de simuler lintelligence , selon le Larousse. Soit des ordinateurs ou des programmes avec des puissances de calcul capables de performances habituellement associes  lintelligence humaine, et amplifies par la technologie : \n- Capacit de traiter de grandes quantits de donnes\n- Facult de discerner des patterns et des modles indtectables par un humain\n- Aptitude  comprendre et analyser ces modles\n- Capacits  interagir avec lhomme\n

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [ ]:
reader = FARMReader(model_name_or_path="etalab-ia/camembert-base-squadFR-fquad-piaf", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'etalab-ia/camembert-base-squadFR-fquad-piaf' (Camembert)
INFO:haystack.modeling.model.language_model:Auto-detected model language: french
INFO:haystack.modeling.model.language_model:Loaded 'etalab-ia/camembert-base-squadFR-fquad-piaf' (Camembert model) from model hub.
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
# You can configure how many candidates the Reader and Retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    query="Qu'est ce que intelligence artificiel?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

In [ ]:
from pprint import pprint

pprint(prediction['answers'])

[<Answer {'answer': 'les ordinateurs et programmes informatiques capables de performances habituellement associes  lintelligence humaine', 'type': 'extractive', 'score': 0.8493160605430603, 'context': ' dIA pour dsigner les ordinateurs et programmes informatiques capables de performances habituellement associes  lintelligence humaine. Par exemple, la', 'offsets_in_document': [{'start': 67, 'end': 182}], 'offsets_in_context': [{'start': 18, 'end': 133}], 'document_id': '84bda6f6d8da8c09c50201192f44b1fa', 'meta': {'name': 'textAI.txt'}}>,
 <Answer {'answer': "un  ensemble de thories et de techniques mises en uvre en vue de raliser des machines capables de simuler l'intelligence humaine", 'type': 'extractive', 'score': 0.7839241623878479, 'context': "e (IA) est un  ensemble de thories et de techniques mises en uvre en vue de raliser des machines capables de simuler l'intelligence humaine 1.\nElle en", 'offsets_in_document': [{'start': 37, 'end': 165}], 'offsets_in_context': [{'start': 11,

In [ ]:
from haystack.pipelines import DocumentSearchPipeline
pipeline = DocumentSearchPipeline(retriever)
query = 'IA '
result = pipeline.run(query, params={"Retriever": {"top_k": 10}})


In [ ]:
from haystack.utils import print_documents

print_documents(result, max_text_len=100, print_name=True, print_meta=True)


Query: IA 

{   'content': "L'intelligence artificielle (IA) est un  ensemble de thories "
               'et de techniques mises en uvre en vue d...',
    'meta': {'name': 'AI.txt'},
    'name': 'AI.txt'}



In [ ]:
[x.to_dict() for x in result["documents"]]

[{'content': "L'intelligence artificielle (IA) est un  ensemble de thories et de techniques mises en uvre en vue de raliser des machines capables de simuler l'intelligence humaine 1.\nElle englobe donc un ensemble de concepts et de technologies, plus qu'une discipline autonome constitue2. Des instances, telle la CNIL, notant le peu de prcision de la dfinition de l'IA, l'ont prsente comme  le grand mythe de notre temps 3.\nSouvent classe dans le groupe des mathmatiques et des sciences cognitives, elle fait appel  la neurobiologie computationnelle (particulirement aux rseaux neuronaux) et  la logique mathmatique (partie des mathmatiques et de la philosophie). Elle utilise des mthodes de rsolution de problmes  forte complexit logique ou algorithmique. Par extension, elle comprend, dans le langage courant, les dispositifs imitant ou remplaant l'homme dans certaines mises en uvre de ses fonctions cognitives4.\nSes finalits et enjeux ainsi que son dveloppement suscitent, depuis l'apparition 